## <font color=blue>Check Individual Module *AR5K14* <font color=green> **local** </font>  Results</font>

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import netCDF4 as nc
from pandas.plotting import table 
import xarray as xr

#### <font color=blue>List of *Modules* and <font color=green> **Datasets** </font> to be used </font>

In [2]:
# Load FACTS workflow data as a dataframe.
df = pd.read_fwf('../001_GMD_IP_Modules_Data/ModuleTest_DataFile_List_ar5k14.local.txt',comment = '#')
df

,Component,Module,sub_Module,DataFile
0,TE,kopp14,sterodynamics,k14sterodynamics.kopp14.sterodynamics_localsl.nc
1,TE,ipccar5,thermalexpansion,ar5TE.ipccar5.thermalexpansion_localsl.nc
2,Glaciers,kopp14,glaciers,k14glaciers.kopp14.glaciers_localsl.nc
3,Glaciers,ipccar5,glaciers,ar5glaciers.ipccar5.glaciers_localsl.nc
4,GrIS,ipccar5,icesheets,ar5icesheets.ipccar5.icesheets_GIS_localsl.nc
5,GrIS,kopp14,icesheets,k14icesheets.kopp14.icesheets_GIS_localsl.nc
6,AIS,deconto16,AIS,deconto16.deconto16.AIS_AIS_localsl.nc
7,EAIS,ipccar5,icesheets,ar5icesheets.ipccar5.icesheets_EAIS_localsl.nc
8,EAIS,deconto16,AIS,deconto16.deconto16.AIS_EAIS_localsl.nc
9,EAIS,kopp14,icesheets,k14icesheets.kopp14.icesheets_EAIS_localsl.nc


#### <font color=blue> **OPTIONAL USE**: <font color=green> *Write nc data details to a file* </font>

In [3]:
EXPDATE     = 221209
MODNO       = df.index.values;   # Number of datasets Module outputs
SCENARIO    = ['rcp26', 'rcp45', 'rcp85']; 
#        
file = open("ModuleData_Info_AR5K14.local.txt", "w")
#
for val in MODNO: 
    for scenario in SCENARIO:
        #
        MODULE = df["Module"][val];  SUB_MODULE = df["sub_Module"][val]; COMP = df["Component"][val];
        # Skip if data is absent.
        if df["DataFile"][val] == 'XXX':
           continue 
        #
        # Pick the data files & Import nc file to dataframe. 
        dataFOLDER  = '/projects/kopp/facts-experiments/{arg2}/ar5k14.{arg1}/output/'.format(arg1=scenario,arg2=EXPDATE)
        dataFILE    = 'ar5k14.{arg1}.'.format(arg1=scenario) + df["DataFile"][val]
        d_nc        = xr.open_dataset(dataFOLDER + dataFILE)
        d_nc
    file.write( "DataFILE ==> "+str(df["DataFile"][val])+"\n"+ str(d_nc.dims) + "\n" + str(d_nc.data_vars)+ "\n" + str(d_nc.coords)+"\n"+"-------------"+"\n"+"\n" )
        #
file.close()


#### $\color{Red}{\text{Load Module data from ==> facts-experiments/221209}}$

In [6]:
EXPDATE     = 221209
MODNO       = df.index.values;   # Number of datasets Module outputs
# MODNO       = np.arange(7);
SCENARIO    = ['rcp26', 'rcp45', 'rcp85']; 
YEAR        = 2100; # Percentile Year for the table.    
DATAEXCEPT = ['k14sterodynamics.kopp14.sterodynamics_localsl.nc',
              'ar5TE.ipccar5.thermalexpansion_localsl.nc',
              'k14glaciers.kopp14.glaciers_localsl.nc',
              'k14icesheets.kopp14.icesheets_GIS_localsl.nc',
              'deconto16.deconto16.AIS_AIS_localsl.nc',
              'deconto16.deconto16.AIS_EAIS_localsl.nc',
              'k14icesheets.kopp14.icesheets_EAIS_localsl.nc',
              'deconto16.deconto16.AIS_WAIS_localsl.nc',
              'k14icesheets.kopp14.icesheets_WAIS_localsl.nc']
            
#
# Temp DF to store all .nc O/P.      
df_TEMP      = []
#        
for scenario in SCENARIO:
    # Define pd DF columns to append.
    PERCENTILE  = '%ile({arg2})'.format(arg2=scenario)
    percentile = np.nan
    df[PERCENTILE] = percentile
    df[PERCENTILE] = df[PERCENTILE].astype('object') 
    #
    for val in MODNO: 
        #
        MODULE = df["Module"][val];  SUB_MODULE = df["sub_Module"][val]; COMP = df["Component"][val]; DATAFILE = df["DataFile"][val];
        # Skip if data is absent.
        if df["DataFile"][val] == 'XXX':
           continue 
        #
        # Pick the data files & Import nc file to dataframe. 
        dataFOLDER  = '/projects/kopp/facts-experiments/{arg2}/ar5k14.{arg1}/output/'.format(arg1=scenario,arg2=EXPDATE)
        dataFILE    = 'ar5k14.{arg1}.'.format(arg1=scenario) + df["DataFile"][val]
        d_nc        = xr.open_dataset(dataFOLDER + dataFILE)
        d_nc
        #
        # Percentile calculation.
        percentList = [50, 17, 83]          #[50, 5, 17, 83, 95]
        # Find year index to pick SLC value
        Yind        = np.where(d_nc["years"].values == YEAR)[0][0]
        #
        if any(x in DATAFILE for x in DATAEXCEPT):
            GMSL    = (d_nc["localSL_quantiles"][:,0,Yind].values)/1

        else:
            GMSL    = (d_nc["sea_level_change"][:,Yind,0].values)/1       
        # Find Percentiles.
        pcntle = np.percentile(GMSL[:], percentList );    pcntle = np.around(pcntle,2)
        #
        # df.at[val,PERCENTILE] = [pcntle[0], pcntle[1],pcntle[2]]
        df.at[val,PERCENTILE] = f'{pcntle[0]:2.2f} ({pcntle[1]:2.2f} , {pcntle[2]:2.2f})' 
        # Print the OP.
print('GMSL Component Projections for ',d_nc["years"][Yind].values)    
#show(df)
df

GMSL Component Projections for  2100


,Component,Module,sub_Module,DataFile,%ile(rcp26),%ile(rcp45),%ile(rcp85)
0,TE,kopp14,sterodynamics,k14sterodynamics.kopp14.sterodynamics_localsl.nc,"199.00 (-32.67 , 435.10)","313.50 (39.52 , 591.72)","481.50 (118.90 , 848.96)"
1,TE,ipccar5,thermalexpansion,ar5TE.ipccar5.thermalexpansion_localsl.nc,"0.00 (0.00 , 0.00)","0.00 (0.00 , 0.00)","0.00 (0.00 , 0.00)"
2,Glaciers,kopp14,glaciers,k14glaciers.kopp14.glaciers_localsl.nc,"89.50 (69.19 , 113.81)","102.50 (77.19 , 128.81)","136.00 (107.19 , 168.00)"
3,Glaciers,ipccar5,glaciers,ar5glaciers.ipccar5.glaciers_localsl.nc,"72.50 (39.83 , 112.00)","91.00 (59.00 , 136.00)","126.00 (80.83 , 183.00)"
4,GrIS,ipccar5,icesheets,ar5icesheets.ipccar5.icesheets_GIS_localsl.nc,"33.00 (23.83 , 43.00)","41.50 (28.00 , 53.00)","71.00 (49.66 , 90.00)"
5,GrIS,kopp14,icesheets,k14icesheets.kopp14.icesheets_GIS_localsl.nc,"26.00 (14.19 , 49.62)","37.00 (14.19 , 66.62)","59.50 (31.00 , 112.43)"
6,AIS,deconto16,AIS,deconto16.deconto16.AIS_AIS_localsl.nc,"730.00 (463.71 , 1496.00)","730.00 (463.71 , 1496.00)","730.00 (463.71 , 1496.00)"
7,EAIS,ipccar5,icesheets,ar5icesheets.ipccar5.icesheets_EAIS_localsl.nc,"11.00 (-2.00 , 21.00)","10.00 (-3.00 , 20.00)","8.00 (-6.00 , 18.00)"
8,EAIS,deconto16,AIS,deconto16.deconto16.AIS_EAIS_localsl.nc,"208.50 (-9.24 , 830.00)","208.50 (-9.24 , 830.00)","208.50 (-9.24 , 830.00)"
9,EAIS,kopp14,icesheets,k14icesheets.kopp14.icesheets_EAIS_localsl.nc,"-10.00 (-55.81 , 95.86)","-12.50 (-60.81 , 92.24)","-19.50 (-71.81 , 91.05)"


In [7]:
df.to_excel(r'Table_CheckModule_AR5K14_local_GMSL-Proj_2100.xlsx', index=False)
